# SDSC3001 - Assignment 3


## Question 1

In [ ]:
# sampling algorithm

# proof of correctness

## Question 2

### Part A

When an itemset $I$ has a size of $m$, there are $2^m -1$ possible subsets. When mining top-$k$ most frequent patterns, 

$$
2^m - 1 \leq k \\
2^m \leq k + 1 \\
m \leq \log_2(k + 1) \\
\therefore m = \lceil \log_2(k + 1) \rceil
$$

### Part B

In [ ]:
import numpy as np
import polar as pl
def load_data():
    with open("trans.txt") as f:
        lines = f.readlines()
        for i in range(len(lines)):
            lines[i] = lines[i].strip().split()
            lines[i] = [int(x) for x in lines[i]]

In [ ]:
# Misra-Greis Algorithm

#### b.1

#### b.2

#### b.3

#### b.4